In [1]:
import os
import glob

import tqdm
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
from tensorflow.contrib.framework import list_variables, load_variable

from tacotron.models import Tacotron
from hparams import hparams

In [2]:
checkpoint_dir = 'logs-Tacotron/taco_pretrained_r9y9'
assert os.path.isdir(checkpoint_dir)

In [3]:
# source : https://gist.github.com/batzner/7c24802dd9c5e15870b4b56e22135c96

checkpoint = tf.train.get_checkpoint_state(checkpoint_dir)

dry_run = False

variables = list_variables(checkpoint_dir)

with tf.Session() as sess, tqdm.tqdm(total=len(variables)) as pbar:
    for var_name, _ in list_variables(checkpoint_dir):
        # Load the variable
        var = load_variable(checkpoint_dir, var_name)

        # Set the new name
        new_name = var_name

        new_name = new_name.replace('model', 'Tacotron_model')
        new_name = new_name.replace('attention_variable', 'attention_variable_projection')
        for i in range(hparams.decoder_layers):
            new_name = new_name.replace(f'decoder_lstm/multi_rnn_cell/cell_{i}/ZoneoutLSTMCell/Linear/Bias',
                                        f'decoder_LSTM/multi_rnn_cell/cell_{i}/decoder_LSTM_{i+1}/bias')
            new_name = new_name.replace(f'decoder_lstm/multi_rnn_cell/cell_{i}/ZoneoutLSTMCell/Linear/Matrix',
                                        f'decoder_LSTM/multi_rnn_cell/cell_{i}/decoder_LSTM_{i+1}/kernel')
        new_name = new_name.replace('linear_transform', 'linear_transform_projection')
        for d in ('fw', 'bw'):
            new_name = new_name.replace(f'encoder_LSTM/bidirectional_rnn/{d}/ZoneoutLSTMCell/Linear/Bias',
                                        f'encoder_LSTM/bidirectional_rnn/{d}/encoder_{d}_LSTM/bias')
            new_name = new_name.replace(f'encoder_LSTM/bidirectional_rnn/{d}/ZoneoutLSTMCell/Linear/Matrix',
                                        f'encoder_LSTM/bidirectional_rnn/{d}/encoder_{d}_LSTM/kernel')
            
        if dry_run:
#             print('%s would be renamed to %s' % (var_name, new_name))
            pass
        else:
#             print('Renaming %s to %s' % (var_name, new_name))
            # Rename the variable
            var = tf.Variable(var, name=new_name)
            
        pbar.update(1)

    if not dry_run:
        # Save the variables
        saver = tf.train.Saver()
        sess.run(tf.global_variables_initializer())
        new_checkpoint_path = saver.save(sess, checkpoint.model_checkpoint_path)
        
#         *previous, last_step = checkpoint.model_checkpoint_path.split('-')
#         last_step = str(int(last_step) + 1)
#         new_checkpoint_path = '-'.join((*previous, last_step))
        
#         old_files = glob.glob(new_checkpoint_path + '*')
#         for f in old_files:
#             os.remove(f)
        
#         new_checkpoint_path = saver.save(sess, new_checkpoint_path)

print(new_checkpoint_path)

100%|████████████████████████████████████████████████████████████████████| 190/190 [00:21<00:00,  8.70it/s]

logs-Tacotron/taco_pretrained_r9y9\model.ckpt-189500


In [4]:
# variables = list_variables(os.path.dirname(new_checkpoint_path))
# for v, _ in variables:
#     print(v)

In [5]:
inputs = tf.placeholder(tf.int32, (None, None), name='inputs')
input_lengths = tf.placeholder(tf.int32, (None), name='input_lengths')
# targets = tf.placeholder(tf.float32, (None, None, hparams.num_mels), name='mel_targets')
split_infos = tf.placeholder(tf.int32, shape=(hparams.tacotron_num_gpus, None), name='split_infos')
with tf.variable_scope('Tacotron_model', reuse=tf.AUTO_REUSE) as scope:
    model = Tacotron(hparams)
    model.initialize(inputs, input_lengths, split_infos=split_infos)
    
    variables = tf.trainable_variables()
    
#     for v in variables:
#         print(v.name)

initialisation done /gpu:0
Initialized Tacotron model. Dimensions (? = dynamic shape): 
  Train mode:               False
  Eval mode:                False
  GTA mode:                 False
  Synthesis mode:           True
  Input:                    (?, ?)
  device:                   0
  embedding:                (?, ?, 512)
  enc conv out:             (?, ?, 512)
  encoder out:              (?, ?, 512)
  decoder out:              (?, ?, 80)
  residual out:             (?, ?, 512)
  projected residual out:   (?, ?, 80)
  mel out:                  (?, ?, 80)
  <stop_token> out:         (?, ?)
  Tacotron Parameters       108.945 Million.
